In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop3.2.tgz
!tar -xvf spark-3.0.0-preview2-bin-hadoop3.2.tgz
!pip install -q findspark

spark-3.0.0-preview2-bin-hadoop3.2/
spark-3.0.0-preview2-bin-hadoop3.2/data/
spark-3.0.0-preview2-bin-hadoop3.2/data/streaming/
spark-3.0.0-preview2-bin-hadoop3.2/data/streaming/AFINN-111.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_binary_classification_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_kmeans_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_multiclass_classification_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_lda_libsvm_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/iris_libsvm.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/pagerank_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_linear_regression_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/pic_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/als/
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/als/test.data
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/als/sample_movielens_rati

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop3.2"

In [0]:
import findspark
findspark.init("spark-3.0.0-preview2-bin-hadoop3.2")# SPARK_HOME

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
df = spark.createDataFrame([{"hello": "world"} for x in range(1000)])
df.show(3)

spark-3.0.0-preview2-bin-hadoop3.2/python/pyspark/sql/session.py:375: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+-----+
|hello|
+-----+
|world|
|world|
|world|
+-----+
only showing top 3 rows



In [0]:
spark.version

'3.0.0-preview2'

In [0]:
from google.colab import files
uploaded = files.upload()

Saving test_2v.csv to test_2v.csv
Saving train_2v.csv to train_2v.csv


In [0]:
train = spark.read.csv('train_2v.csv',inferSchema=True,header=True)

In [0]:
train.printSchema()

root
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [0]:
train.groupBy('stroke').count().show()

+------+-----+
|stroke|count|
+------+-----+
|     0|42617|
|     1|  783|
+------+-----+



In [0]:
train.createOrReplaceTempView('table')

In [0]:
spark.sql("SELECT work_type, count(work_type) as work_type_count FROM table WHERE stroke == 1 GROUP BY work_type ORDER BY work_type_count DESC").show()

+-------------+---------------+
|    work_type|work_type_count|
+-------------+---------------+
|      Private|            441|
|Self-employed|            251|
|     Govt_job|             89|
|     children|              2|
+-------------+---------------+



In [0]:
spark.sql("SELECT gender, count(gender) as count_gender, count(gender)*100/sum(count(gender)) over() as percent FROM table GROUP BY gender").show()

+------+------------+-------------------+
|gender|count_gender|            percent|
+------+------------+-------------------+
|  Male|       17724|  40.83870967741935|
|Female|       25665|  59.13594470046083|
| Other|          11|0.02534562211981567|
+------+------------+-------------------+



In [0]:
spark.sql("SELECT gender, count(gender), (COUNT(gender) * 100.0) /(SELECT count(gender) FROM table WHERE gender == 'Male') as percentage FROM table WHERE stroke = '1' and gender = 'Male' GROUP BY gender").show()

+------+-------------+----------------+
|gender|count(gender)|      percentage|
+------+-------------+----------------+
|  Male|          352|1.98600767321146|
+------+-------------+----------------+



In [0]:
spark.sql("SELECT age, count(age) as age_count FROM table WHERE stroke == 1 GROUP BY age ORDER BY age_count DESC").show()

+----+---------+
| age|age_count|
+----+---------+
|79.0|       70|
|78.0|       57|
|80.0|       49|
|81.0|       43|
|82.0|       36|
|70.0|       25|
|76.0|       24|
|77.0|       24|
|74.0|       24|
|67.0|       23|
|75.0|       23|
|72.0|       21|
|59.0|       20|
|68.0|       20|
|69.0|       20|
|57.0|       19|
|71.0|       19|
|65.0|       18|
|63.0|       18|
|66.0|       17|
+----+---------+
only showing top 20 rows



In [0]:
train_f = train.na.fill('No Info', subset=['smoking_status'])

In [0]:
from pyspark.sql.functions import mean
mean = train_f.select(mean(train_f['bmi'])).collect()
mean_bmi = mean[0][0]
train_f = train_f.na.fill(mean_bmi,['bmi'])

In [0]:
from pyspark.ml.feature import (VectorAssembler,OneHotEncoder,StringIndexer)

In [0]:
gender_indexer = StringIndexer(inputCol='gender',outputCol='genderIndex')
gender_encoder = OneHotEncoder(inputCol='genderIndex',outputCol='genderVec')

In [0]:
ever_married_indexer = StringIndexer(inputCol='ever_married',outputCol='ever_marriedIndex')
ever_married_encoder = OneHotEncoder(inputCol='ever_marriedIndex',outputCol='ever_marriedVec')

In [0]:
Residence_type_indexer = StringIndexer(inputCol='Residence_type',outputCol='Residence_typeIndex')
Residence_type_encoder = OneHotEncoder(inputCol='Residence_typeIndex',outputCol='Residence_typeVec')

In [0]:
smoking_status_indexer = StringIndexer(inputCol='smoking_status',outputCol='smoking_statusIndex')
smoking_status_encoder = OneHotEncoder(inputCol='smoking_statusIndex',outputCol='smoking_statusVec')

In [0]:
work_type_indexer = StringIndexer(inputCol='work_type',outputCol='work_typeIndex')
work_type_encoder = OneHotEncoder(inputCol='work_typeIndex',outputCol='work_typeVec')

In [0]:
assembler = VectorAssembler(inputCols=['genderVec',
 'age',
 'hypertension',
 'heart_disease',
 'ever_marriedVec',
 'work_typeVec',
 'Residence_typeVec',
 'avg_glucose_level',
 'bmi',
 'smoking_statusVec'],outputCol='features')

In [0]:
from pyspark.ml.classification import DecisionTreeClassifier
dtc = DecisionTreeClassifier(labelCol='stroke',featuresCol='features')

In [0]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[gender_indexer, ever_married_indexer, work_type_indexer, Residence_type_indexer,
                           smoking_status_indexer, gender_encoder, ever_married_encoder, work_type_encoder,
                           Residence_type_encoder, smoking_status_encoder, assembler, dtc])

In [0]:
train_data,test_data = train_f.randomSplit([0.7,0.3])
train_data.show()

+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+------------------+---------------+------+
| id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level|               bmi| smoking_status|stroke|
+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+------------------+---------------+------+
|  1|Female|80.0|           0|            1|         Yes|Self-employed|         Urban|            67.45|              25.6|   never smoked|     0|
|  4|Female|37.0|           0|            0|         Yes|      Private|         Rural|            82.15|              24.3|         smokes|     0|
|  6|Female|21.0|           0|            0|          No|      Private|         Urban|            75.73|28.605038390004545|   never smoked|     0|
|  7|Female|44.0|           0|            0|         Yes|      Private|         Urban|           222.75|              

In [0]:
model = pipeline.fit(train_data)

In [0]:
dtc_predictions = model.transform(test_data)

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Select (prediction, true label) and compute test error
acc_evaluator = MulticlassClassificationEvaluator(labelCol="stroke", predictionCol="prediction", metricName="accuracy")
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
print('A Decision Tree algorithm had an accuracy of: {0:2.2f}%'.format(dtc_acc*100))

A Decision Tree algorithm had an accuracy of: 98.22%
